# PyTorch 基本运算示例

Torch的tensor和Numpy的Array，Pandas的Series和Dataframe类似，是了解应用这些代码库的基础，这里首先简单快速地了解Tensor及如何使用GPU，然后日常积累一些常用张量运算函数。

主要参考：

- [TENSORS](https://pytorch.org/tutorials/beginner/former_torchies/tensor_tutorial.html)
- [Speed Up your Algorithms Part 1 — PyTorch](https://towardsdatascience.com/speed-up-your-algorithms-part-1-pytorch-56d8a4ae7051)
- [PyTorch 101, Part 1: Understanding Graphs, Automatic Differentiation and Autograd](https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/)

## 快速了解Tensor

pytorch作为NumPy的替代品，可以利用GPU的性能进行计算；可作为一个高灵活性、速度快的深度学习平台。

Tensor（张量）类似于NumPy的ndarray，但还可以在GPU上使用来加速计算。因此经常看到把numpy的数组包装为tensor再运算。tensor的操作和numpy中的数组操作类似，不再赘述，详见官网。下面列举一些简单例子。首先pytorch的导入是import torch，因为torch一直都是那个torch，一开始是别的语言写的，现在在python下，所以就叫pytorch。

In [1]:
import torch

Tensor是pytorch的基本数据类型：

In [2]:
x = torch.Tensor(5)
# 如果想要从tensor中获取到长度的int数据
type(list(x.shape)[0])

int

In [3]:
# 构建一个 5x3 的矩阵, 未初始化的:
x = torch.Tensor(5, 3)
x

tensor([[9.9184e-39, 8.7245e-39, 9.2755e-39],
        [8.9082e-39, 9.9184e-39, 8.4490e-39],
        [9.6429e-39, 1.0653e-38, 1.0469e-38],
        [4.2246e-39, 1.0378e-38, 9.6429e-39],
        [9.2755e-39, 9.7346e-39, 1.0745e-38]])

pytorch中的一些基本运算：

In [4]:
# 构建一个随机初始化的矩阵:
x = torch.rand(5, 3)
x

tensor([[0.4398, 0.1398, 0.7243],
        [0.2603, 0.5517, 0.9940],
        [0.8248, 0.4888, 0.6620],
        [0.1417, 0.2469, 0.4292],
        [0.4014, 0.0369, 0.4385]])

In [5]:
x.size()

torch.Size([5, 3])

concatenate的操作使用torch.cat可以完成

In [6]:
x1 = torch.Tensor(1)
x2 = torch.Tensor(1)
torch.cat((x1,x2),0)

tensor([0.0000e+00, 8.5451e-07])

可以看到torch中的size也是torch中的类，包装了python的list，自然地，加减运算的对象也都是torch的tensor了。运算可以使用运算符，也可以使用函数。

In [7]:
# 加法
y = torch.rand(5, 3)
x + y

tensor([[0.8916, 0.9551, 1.1712],
        [0.5677, 0.5823, 1.0931],
        [1.4513, 1.4779, 1.5074],
        [0.8993, 0.2620, 0.7302],
        [1.3191, 0.1133, 1.2089]])

In [8]:
torch.add(x, y)

tensor([[0.8916, 0.9551, 1.1712],
        [0.5677, 0.5823, 1.0931],
        [1.4513, 1.4779, 1.5074],
        [0.8993, 0.2620, 0.7302],
        [1.3191, 0.1133, 1.2089]])

In [9]:
result = torch.Tensor(5, 3)
torch.add(x, y, out = result)

tensor([[0.8916, 0.9551, 1.1712],
        [0.5677, 0.5823, 1.0931],
        [1.4513, 1.4779, 1.5074],
        [0.8993, 0.2620, 0.7302],
        [1.3191, 0.1133, 1.2089]])

In [10]:
a = torch.ones(5)
a.add_(1)

tensor([2., 2., 2., 2., 2.])

numpy和tensor之间有很多类似的地方，比如索引，形状改变等：

In [11]:
# 可以用类似Numpy的索引来处理所有的张量！
x[:, 1]

tensor([0.1398, 0.5517, 0.4888, 0.2469, 0.0369])

In [12]:
# 改变大小: 如果你想要去改变tensor的大小, 可以使用 torch.view:
x = torch.randn(4, 4)
y = x.view(16)
z = x.view(-1, 8)  # -1就是让pytorch自己根据其他的维度去判断这里该是几维
print(x.size(), y.size(), z.size())
print(x)
print(y)
print(z)

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])
tensor([[-0.1869, -1.4803,  0.7319, -0.8843],
        [ 0.8513,  0.9019, -0.6537, -0.3310],
        [-0.4791, -0.5095, -0.0829,  0.9046],
        [-0.0829,  0.5256,  0.1591,  0.3896]])
tensor([-0.1869, -1.4803,  0.7319, -0.8843,  0.8513,  0.9019, -0.6537, -0.3310,
        -0.4791, -0.5095, -0.0829,  0.9046, -0.0829,  0.5256,  0.1591,  0.3896])
tensor([[-0.1869, -1.4803,  0.7319, -0.8843,  0.8513,  0.9019, -0.6537, -0.3310],
        [-0.4791, -0.5095, -0.0829,  0.9046, -0.0829,  0.5256,  0.1591,  0.3896]])


由于和numpy的紧密联系，因此pytorch的张量和numpy数组可以很方便的转换。

In [13]:
a = torch.ones(5)
b = a.numpy()
b

array([1., 1., 1., 1., 1.], dtype=float32)

In [14]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
b

tensor([1., 1., 1., 1., 1.], dtype=torch.float64)

要注意numpy的array和torch的tensor转换后，数据是绑定的，如下所示：

In [15]:
# 看改变 np 数组之后 Torch Tensor 是如何自动改变的
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
print("numpy的array：",a)
print("array转为torch的tensor：",b)
np.add(a, 1, out = a)
print(a)
print(b)

numpy的array： [1. 1. 1. 1. 1.]
array转为torch的tensor： tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


In [16]:
# 可以使用 .cuda 方法将 Tensors 在GPU上运行.
# 只要在  CUDA 是可用的情况下, 我们可以运行这段代码
if torch.cuda.is_available():
    b = b.cuda()
    print(b + b)

C:\Users\11445\miniconda3\envs\hydrus\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


CPU上的所有张量(CharTensor除外)都支持与Numpy的相互转换。

张量要在GPU上计算，需要主动从CPU移动到GPU上。张量可以使用.to方法移动到任何设备（device）上

In [17]:
x = torch.randn(1)
print(x)
print(x.item())
# 当GPU可用时,我们可以运行以下代码
# 我们将使用`torch.device`来将tensor移入和移出GPU
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # 直接在GPU上创建tensor
    x = x.to(device)                       # 或者使用`.to("cuda")`方法
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))       # `.to`也能在移动时改变dtype

tensor([0.9909])
0.9909061193466187


## GPU or CPU

检查 cuda 设备是否可用：

In [1]:
import torch
torch.cuda.is_available()

C:\Users\11445\miniconda3\envs\hydrus\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


False

In [3]:
# 如果有GPU的话 可以执行下面几个注释掉的代码
# torch.cuda.current_device()

In [5]:
# torch.cuda.get_device_name(0)

In [7]:
# Returns the current GPU memory usage by tensors in bytes for a given device
# torch.cuda.memory_allocated()

In [8]:
# Returns the current GPU memory managed by the caching allocator in bytes for a given device
# torch.cuda.memory_reserved()

In [8]:
# Releases all unoccupied cached memory currently held by the caching allocator so that those can be used in other GPU application and visible in nvidia-smi
# torch.cuda.empty_cache()

如何使用GPU？

如果像存储数据到CPU，那么简单定义即可：

In [9]:
a = torch.DoubleTensor([1., 2.])

这时候变量在CPU上，且后续的运算都会在CPU上，为了将数据转移到GPU上，需要使用 .cuda

In [10]:
# a = torch.FloatTensor([1., 2.]).cuda()

或者

In [11]:
# a = torch.cuda.FloatTensor([1., 2.])

这时候数据就在GPU上。模型同样可以转移到GPU上，比如：

In [14]:
from torch import nn
model = nn.Sequential(
         nn.Linear(20, 20),
         nn.ReLU(),
         nn.Linear(20, 4),
         nn.Softmax()
)
# model = model.cuda()

检查在不在GPU上。

In [15]:
next(model.parameters()).is_cuda

False

如果有多个GPU，可以使用Data Parallelism，拆分数据，将 Data Generator 拆分到更小的mini batches，然后送到多个GPUs

PyTorch中，并行通过torch.nn.DataParallel实现。一些关键函数：：

- Replicate：Module在多个设备上复制。
- Scatter：input在这些设备的第一维中分布。
- Gather：从这些设备收集input和连接第一维。
- parallel_apply：将从Scatter获得的一组输入分布式应用于从Replicate获得的相应的分布式Modules。

```Python
# Replicate module to devices in device_ids
replicas = nn.parallel.replicate(module, device_ids)
# Distribute input to devices in device_ids
inputs = nn.parallel.scatter(input, device_ids)
# Apply the models to corresponding inputs
outputs = nn.parallel.parallel_apply(replicas, inputs)
# Gather result from all devices to output_device
result = nn.parallel.gather(outputs, output_device)
```

或者一种更简单的方式：

```Python
model = nn.DataParallel(model, device_ids=device_ids)
result = model(input)
```

更多信息可以关注：

- [Multi-GPU Framework Comparisons](https://medium.com/@iliakarmanov/multi-gpu-rosetta-stone-d4fa96162986)
- [ilkarman/DeepLearningFrameworks](https://github.com/ilkarman/DeepLearningFrameworks)

下面是Tensor的常见计算，日常积累。

## 基本算术运算

Tensor是一种数据结构，是 PyTorch 的基本构建块。Tensors 非常像 numpy 数组，与 numpy 不同的是，张量旨在利用 GPU 的并行计算能力。许多 Tensor 语法类似于 numpy 数组的语法。

In [45]:
import numpy as np
import torch

### 初始化

直接调用Tensor即可完成初始化。

In [54]:
tt = torch.Tensor(3)

和numpy不同的是，为了完成梯度下降等优化计算，Tensor还有梯度相关的属性（后续文本会有更多介绍），初始化时候requires_grad默认是False

In [55]:
tt.requires_grad

False

初始化为0

In [52]:
torch.zeros(4)

tensor([0., 0., 0., 0.])

Tensor

In [53]:
t_one = torch.ones(4)
t_one.requires_grad

False

requires_grad具有传染性。这意味着当通过对其他Tensors运算创建一个Tensor，且其中至少有一个用于创建的张量requires_grad被设置为True时，运算结果Tensor的requires_grad也将被设置为True。

In [57]:
a = torch.randn((3,3), requires_grad = True)
w1 = torch.randn((3,3))
b = w1*a 
b.requires_grad

True

如果想要计算后的张量不被前序变量传染，可以使用detach()（非inplace运算），先detach()再计算即可。

In [67]:
b_detach = w1*(a.detach())
b_detach == b

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])

In [68]:
b_detach.requires_grad

False

detach()不是inplace运算，所以a还是有requires_grad的。

In [69]:
a.requires_grad

True

### 均值计算

In [16]:
X = np.random.rand(6,4)
print(X,X.shape)
avg_np, _ = np.average(X, axis=0, returned=True)
avg_np, _

[[0.44499848 0.06677202 0.48286518 0.79960818]
 [0.55011906 0.84638496 0.9129834  0.72624932]
 [0.68279485 0.72864658 0.56643253 0.85884824]
 [0.01584725 0.83010497 0.17834428 0.8620229 ]
 [0.8497338  0.66295321 0.10840663 0.52652553]
 [0.22564929 0.36593341 0.9118287  0.71547575]] (6, 4)


(array([0.46152379, 0.58346586, 0.52681012, 0.74812165]),
 array([6., 6., 6., 6.]))

In [17]:
X_th = torch.tensor(X)
avg_th = torch.mean(X_th, dim=0)
avg_th

tensor([0.4615, 0.5835, 0.5268, 0.7481], dtype=torch.float64)

In [18]:
assert (X == X_th.numpy()).all()
# assert (avg_np == avg_th.numpy()).all()   # this fails alread

### 求和计算

求和运算和平均运算类似：

In [19]:
ta=torch.ones(5)
tc=torch.sum(ta)
tc

tensor(5.)

### PyTorch中的广播

试试torch的广播功能：

In [20]:
t1 = torch.FloatTensor([[1,2,3],[4,5,6]])
t2=torch.sqrt(t1)

In [21]:
t1/((t2+0.1)**2)

tensor([[0.8264, 0.8723, 0.8938],
        [0.9070, 0.9162, 0.9231]])

### element-wise运算

In [31]:
ta=torch.ones(5)
tb=torch.zeros(5)
ta-tb

tensor([1., 1., 1., 1., 1.])

In [32]:
z = torch.FloatTensor([2]).repeat(1,5)
z

tensor([[2., 2., 2., 2., 2.]])

In [33]:
z/ta

tensor([[2., 2., 2., 2., 2.]])

In [34]:
td=ta/z
td

tensor([[0.5000, 0.5000, 0.5000, 0.5000, 0.5000]])

In [35]:
te=td**2
te

tensor([[0.2500, 0.2500, 0.2500, 0.2500, 0.2500]])

看看二维的情况：

In [36]:
t1 = torch.FloatTensor([[1,2,3],[4,5,6]])
t2=t1.repeat(1,4).view(-1, 3)
t2

tensor([[1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.]])

In [37]:
t3 = torch.FloatTensor([[7,8,9],[10,11,12]])
t4=t3.repeat(1,4).view(-1, 3)
t4

tensor([[ 7.,  8.,  9.],
        [ 7.,  8.,  9.],
        [ 7.,  8.,  9.],
        [ 7.,  8.,  9.],
        [10., 11., 12.],
        [10., 11., 12.],
        [10., 11., 12.],
        [10., 11., 12.]])

In [38]:
SST = (t4 - t2) ** 2
SST

tensor([[36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.],
        [36., 36., 36.]])

In [39]:
SST=torch.sum(SST,dim=0)
SST

tensor([288., 288., 288.])

### 比较大小

两个张量，element-wise比较大小

In [43]:
a = torch.tensor((1, 2, -1))
b = torch.tensor((3, 0, 4))
torch.maximum(a, b)

tensor([3, 2, 4])

如果张量大小不同，也会默认执行广播运算

In [44]:
a = torch.tensor((1, 2, -1))
b = torch.tensor([0])
torch.maximum(a, b)

tensor([1, 2, 0])

### 处理NaN值

有时候会有一nan值需要处理。在torch中检测是否有nan值可以使用：x != x

In [22]:
print(np.nan==np.nan)

False


In [23]:
t5 = torch.FloatTensor([[7,np.nan,9],[10,11,12]])
t5

tensor([[ 7., nan,  9.],
        [10., 11., 12.]])

如果是直接判断一个多维张量是否有nan值，可以使用：

In [40]:
mask= t5==t5
mask

tensor([[ True, False,  True],
        [ True,  True,  True]])

In [41]:
if len(mask[mask == True]) > 0:
    print("please check")

please check


In [25]:
t6=t5[mask]
t6

tensor([ 7.,  9., 10., 11., 12.])

为了让一般的数值计算能够成为梯度可追踪的计算，我们有时候需要将常见的一些计算用tensor重写，下面是NSE的例子：

In [26]:
def nse_2d(t,p):
    seq_length = t.shape[0]
    Ngage = t.shape[1]
    tmean = torch.mean(t, dim=0)
    tmeans = tmean.repeat(seq_length, 1)
    SST = torch.sum((t - tmeans) ** 2, dim=0)
    SSRes = torch.sum((t - p) ** 2, dim=0)
    # Same as Fredrick 2019
    # temp = SSRes / ((torch.sqrt(SST) + 0.1) ** 2)
    # original NSE
    temp = SSRes / SST
    loss = torch.sum(temp) / Ngage
    return loss

t1 = torch.FloatTensor([[1,2,3],[4,5,6]])
t2 = torch.FloatTensor([[1,2,3],[4,5,6]])
print(nse_2d(t1,t2))

tensor(0.)


## 对Tensor的操作

In [42]:
import torch

### 复制操作

类似numpy的repeat和tile，torch中可以使用repeat：

In [27]:
z = torch.FloatTensor([[1,2,3],[4,5,6]])
z.repeat(1,4)

tensor([[1., 2., 3., 1., 2., 3., 1., 2., 3., 1., 2., 3.],
        [4., 5., 6., 4., 5., 6., 4., 5., 6., 4., 5., 6.]])

In [28]:
z.repeat(1,4).view(-1, 3)

tensor([[1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [1., 2., 3.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.],
        [4., 5., 6.]])

In [29]:
z.repeat(4,1)

tensor([[1., 2., 3.],
        [4., 5., 6.],
        [1., 2., 3.],
        [4., 5., 6.],
        [1., 2., 3.],
        [4., 5., 6.],
        [1., 2., 3.],
        [4., 5., 6.]])

In [1]:
x = torch.randn(2, 3, 4)
x

tensor([[[ 0.3281, -0.8897,  0.7370, -0.9295],
         [ 0.7451,  0.3618, -0.4559, -0.7497],
         [-0.9598, -1.1446, -2.7180, -1.1157]],

        [[ 1.1102, -1.1062, -0.7549,  1.4117],
         [ 0.5135, -1.6110,  0.4967,  1.1779],
         [ 0.5094,  1.6470,  0.0407,  0.1715]]])

In [2]:
x[-1,:,:]

tensor([[ 1.1102, -1.1062, -0.7549,  1.4117],
        [ 0.5135, -1.6110,  0.4967,  1.1779],
        [ 0.5094,  1.6470,  0.0407,  0.1715]])

In [3]:
x[-1,:,:].repeat(3,1,1)

tensor([[[ 1.1102, -1.1062, -0.7549,  1.4117],
         [ 0.5135, -1.6110,  0.4967,  1.1779],
         [ 0.5094,  1.6470,  0.0407,  0.1715]],

        [[ 1.1102, -1.1062, -0.7549,  1.4117],
         [ 0.5135, -1.6110,  0.4967,  1.1779],
         [ 0.5094,  1.6470,  0.0407,  0.1715]],

        [[ 1.1102, -1.1062, -0.7549,  1.4117],
         [ 0.5135, -1.6110,  0.4967,  1.1779],
         [ 0.5094,  1.6470,  0.0407,  0.1715]]])

### 与numpy之间的转换

tensor 转换为numpy：

In [1]:
import torch
a = torch.ones(5)
print(a)

tensor([1., 1., 1., 1., 1.])


In [2]:
b = a.numpy()
print(b)

[1. 1. 1. 1. 1.]


In [3]:
a.add_(1)
print(a)
print(b)    # see how the numpy array changed in value

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


注意tensor变化时，array也会跟着变。反过来也一样：

In [4]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)  # see how changing the np array changed the torch Tensor automatically

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


All the Tensors on the CPU except a CharTensor support converting to NumPy and back.

### 维度变换

再看看维度变换，三维变二维：

In [2]:
import torch
t1 = torch.FloatTensor([[[7,8,9],[10,11,12]],[[7,8,9],[10,11,12]]])
t1

tensor([[[ 7.,  8.,  9.],
         [10., 11., 12.]],

        [[ 7.,  8.,  9.],
         [10., 11., 12.]]])

In [4]:
t2=t1.view(-1, 3)
t2

tensor([[ 7.,  8.,  9.],
        [10., 11., 12.],
        [ 7.,  8.,  9.],
        [10., 11., 12.]])

In [5]:
t3=t1.reshape(-1, 3)
t3

tensor([[ 7.,  8.,  9.],
        [10., 11., 12.],
        [ 7.,  8.,  9.],
        [10., 11., 12.]])

reshape 和 view 的区别这里简单补充下，参考了：[Pytorch-reshape与view的区别](https://congluwen.top/2018/12/pytorch_reshape_view/)。Pytorch中reshape()与view()都可以改变Tensor的shape但是有略微的区别.

- view()只可以由torch.Tensor.view()来调用，view():
    - 不改变Tensor数据，改变Tensor的size(即shape)
    - 对于一个将要被view的Tensor，新的size必须与原来的size与stride兼容,即新的维度必须是以下两种情况: 
        - 是原有维度的一个子空间
        - 只跨越原有满足邻接条件，stride[i]=stride[i+1]×size[i+1]的原有维度d,d+1,…,d+k\
    - 否则，在view之前必须调用contiguous()方法，对于该方法的讨论，见[StackOverflow](https://stackoverflow.com/questions/48915810/pytorch-contiguous)
- reshape()可以由torch.reshape(),也可由torch.Tensor.reshape()调用。reshape():
    - 同样也是返回与input数据量相同，但形状不同的tensor
    - 若满足view的条件，则不会copy，若不满足，则会copy

### 张量转置

In [6]:
import torch
x = torch.randn(2, 3)
x

tensor([[-0.0589, -0.2256, -1.2757],
        [-0.5401,  1.4478, -1.2537]])

In [7]:
torch.t(x)

tensor([[-0.0589, -0.5401],
        [-0.2256,  1.4478],
        [-1.2757, -1.2537]])

### 